In [1]:
!pip install git+https://github.com/CSIRO-enviro-informatics/AusPIX_DGGS.git@geojson    

  Cloning https://github.com/CSIRO-enviro-informatics/AusPIX_DGGS.git (to revision geojson) to /tmp/pip-req-build-qh01ilps
  Running command git clone -q https://github.com/CSIRO-enviro-informatics/AusPIX_DGGS.git /tmp/pip-req-build-qh01ilps
  Running command git checkout -b geojson --track origin/geojson
  Switched to a new branch 'geojson'
  Branch 'geojson' set up to track remote branch 'geojson' from 'origin'.
  Created wheel for auspixdggs: filename=auspixdggs-1.0.0-py3-none-any.whl size=66542 sha256=1126e8f97b61d44cfb40b85afd5efa53f1072e3ad6e472b408ae55c581633c80
  Stored in directory: /tmp/pip-ephem-wheel-cache-tg5mhav6/wheels/fb/47/2e/b372479b10426ba7a860a2db22819a2237f731c183a9383fba
Successfully built auspixdggs


In [2]:
!pip install geojson shapely pygeoj matplotlib

In [11]:
import geojson
fname = "SA_RoadsExample_lines.geojson"
geojson_data = None
with open(fname) as json_file:
    geojson_data = geojson.load(json_file)
    

In [15]:
#draw the geometry for the feature on a map
import ipyleaflet as ipy 
import ipywidgets as ipyw
from ipyleaflet import GeoJSON, Map, Marker

x_coord = -34.74
y_coord = 138.78

map1 = ipy.Map(center=[x_coord, y_coord], zoom=12)
label = ipyw.Label(layout=ipyw.Layout(width='100%'))

geo_json1 = GeoJSON(data=geojson_data, style = {'color': 'blue', 'opacity':0.8, 'weight':1.9, 'fillOpacity':0.3})
map1.add_layer(geo_json1)
map1

Map(center=[-34.74, 138.78], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom…

In [16]:
from shapely.geometry import shape, LineString, MultiLineString, Polygon, MultiPolygon
from geojson.utils import coords
from geojson import Feature, FeatureCollection
from auspixdggs.callablemodules.dggs_in_poly_for_geojson_callable import cells_in_poly, get_dggs_cell_bbox
from auspixdggs.callablemodules.dggs_in_line import line_to_DGGS

def bbox(coord_list):
     box = []
     for i in (0,1):
         res = sorted(coord_list, key=lambda x:x[i])
         box.append((res[0][i],res[-1][i]))
     ret = [ box[0][0], box[1][0], box[0][1], box[1][1] ]
     return ret
    
def geojson_to_shape(g):
    return shape(g)
    
def get_cells_in_geojson(geojson, resolution):
    list_cells = []
    for fea in geojson['features']:  # for feature in attribute table
        res_cells = get_cells_in_feature(fea, resolution)
        list_cells = list(list_cells + res_cells)
    return list_cells 

def get_cells_in_feature(fea, resolution, cell_obj=True):
    geom = geojson_to_shape(fea['geometry'])
    curr_coords = list(coords(fea))
    thisbbox = bbox(curr_coords)
    cells = []
    if isinstance(geom, LineString) or isinstance(geom, MultiLineString): 
        res_cells = line_to_DGGS(curr_coords, resolution)  # start at DGGS level 10   
        cells = res_cells
    elif isinstance(geom, Polygon) or  isinstance(geom, MultiPolygon):
        res_cells = cells_in_poly(thisbbox, curr_coords, resolution, return_cell_obj=True)  # start at DGGS level 10    
        print(res_cells)
        cells = [item[0] for item in res_cells]
    else: #try something anyway
        cells = cells_in_poly(thisbbox, curr_coords, resolution, return_cell_obj=True)  # start at DGGS level 10    
        print(res_cells)
        cells = [item[0] for item in res_cells]

    return cells

list_cells = get_cells_in_geojson(geojson_data, 10)    
list_features = []
for cell in list_cells:
    bbox_coords = get_dggs_cell_bbox(cell)
    geom_obj = Polygon(bbox_coords)
    feat = Feature(geometry=geom_obj) 
    list_features.append(feat)

feature_collection = FeatureCollection(list_features)

#uncomment if you want to get a file output
#geojson_dump = geojson.dumps(feature_collection, indent=4, sort_keys=True)
#with open('poly.geojson', 'w') as json_file:
#    data = geojson.dump(feature_collection, json_file, indent=4, sort_keys=True)        

In [18]:
map2 = ipy.Map(center=[x_coord, y_coord], zoom=12)
label = ipyw.Label(layout=ipyw.Layout(width='100%'))

geo_json2 = GeoJSON(data=feature_collection, 
                    style={
                          'color': 'black', 
                          'opacity': 1, 
                          'weight':1, 
                          'fillColor': 'red', 
                          'fillOpacity': 0.3})
map2.add_layer(geo_json2)
map2.add_layer(geo_json1)
map2

Map(center=[-34.74, 138.78], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom…